In [1]:
import nltk
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('data/artigos.txt', 'r') as _file:
    corpus = _file.read().lower()
    
corpus[:500]

'\n\n\nimagem \n\ntemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\npara salvar um novo usuário, várias validações são feitas, como por exemplo: ver se o nome só contém letras, [**o cpf só números**] e ver se o usuário possui no mínimo 18 anos. veja o método que faz essa validação:\n\njava \n\nsuponha agora que eu tenha outra classe, a classe `produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: ver se só contém letras. e aí? vou'

In [3]:
from string import punctuation

punctuation_list = ''.join(list(punctuation))
punctuation_translator = str.maketrans('', '', punctuation_list)

In [4]:
corpus_v1 = corpus.translate(punctuation_translator)
corpus_v1[:500]

'\n\n\nimagem \n\ntemos a seguinte classe que representa um usuário no nosso sistema\n\njava\n\npara salvar um novo usuário várias validações são feitas como por exemplo ver se o nome só contém letras o cpf só números e ver se o usuário possui no mínimo 18 anos veja o método que faz essa validação\n\njava \n\nsuponha agora que eu tenha outra classe a classe produto que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário ver se só contém letras e aí vou criar outro método '

In [5]:
tokens = nltk.tokenize.word_tokenize(corpus_v1)
print(tokens[:100])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', 'java', 'para', 'salvar', 'um', 'novo', 'usuário', 'várias', 'validações', 'são', 'feitas', 'como', 'por', 'exemplo', 'ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', 'o', 'cpf', 'só', 'números', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no', 'mínimo', '18', 'anos', 'veja', 'o', 'método', 'que', 'faz', 'essa', 'validação', 'java', 'suponha', 'agora', 'que', 'eu', 'tenha', 'outra', 'classe', 'a', 'classe', 'produto', 'que', 'contém', 'um', 'atributo', 'nome', 'e', 'eu', 'quero', 'fazer', 'a', 'mesma', 'validação', 'que', 'fiz', 'para', 'o', 'nome', 'do', 'usuário', 'ver', 'se', 'só', 'contém', 'letras', 'e', 'aí', 'vou', 'criar', 'outro', 'método', 'para', 'fazer', 'a', 'mesma', 'validação', 'ou']


In [6]:
tokens_v1 = list(filter(str.isalpha, tokens))
print(tokens_v1[:100])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', 'java', 'para', 'salvar', 'um', 'novo', 'usuário', 'várias', 'validações', 'são', 'feitas', 'como', 'por', 'exemplo', 'ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', 'o', 'cpf', 'só', 'números', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no', 'mínimo', 'anos', 'veja', 'o', 'método', 'que', 'faz', 'essa', 'validação', 'java', 'suponha', 'agora', 'que', 'eu', 'tenha', 'outra', 'classe', 'a', 'classe', 'produto', 'que', 'contém', 'um', 'atributo', 'nome', 'e', 'eu', 'quero', 'fazer', 'a', 'mesma', 'validação', 'que', 'fiz', 'para', 'o', 'nome', 'do', 'usuário', 'ver', 'se', 'só', 'contém', 'letras', 'e', 'aí', 'vou', 'criar', 'outro', 'método', 'para', 'fazer', 'a', 'mesma', 'validação', 'ou', 'criar']


In [7]:
len(tokens_v1)

406583

In [8]:
from collections import Counter

knowledge = Counter(tokens_v1)
total_knowledge = knowledge.total()

In [9]:
def generate_by_insert(word):
    letters = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    return [
        word[:pos] + letter + word[pos:]
        for letter in letters
        for pos in range(len(word) + 1)
    ]


def generate_by_delete(word):
    return [
        word[:pos] + word[pos + 1:]
        for pos in range(len(word) + 1)
    ]


def generate_by_swap(word):
    letters = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    return [
        word[:pos] + letter + word[pos + 1:]
        for letter in letters
        for pos in range(len(word) + 1)
    ]


def generate_by_exchange(word):
    return [
        word[:pos] + word[pos + 1] + word[pos] + word[pos + 2:]
        for pos in range(len(word) - 1)
    ]


def words_generator_1_dist(word):
    generated_words = [
        *generate_by_insert(word),
        *generate_by_delete(word),
        *generate_by_swap(word),
        *generate_by_exchange(word),
    ]
    
    return generated_words


def words_generator_2_dist(generated_words):
    generated_words_2_dist = []

    for word in generated_words:
        generated_words_2_dist += words_generator_1_dist(word)
    
    return generated_words_2_dist


def get_probability(word):
    return knowledge[word] / total_knowledge


def corrector_v1(word):
    generated_words = words_generator_1_dist(word)
    return max(generated_words, key=get_probability)


def corrector_v2(word):
    generated_words = words_generator_1_dist(word)
    generated_words += words_generator_2_dist(generated_words)
    candidates = [word for word in generated_words if word in knowledge]

    if len(candidates):
        correct_word = max(candidates, key=get_probability)
        return correct_word
    
    return None
    

In [10]:
def score(model, X, Y):
    corrected_words = np.array(list(map(lambda x: model(x), X)))
    return '%.2f%%' % (np.sum(corrected_words == Y) * 100 / X.shape[0])

In [11]:
with open('data/palavras.txt', 'r') as _file:
    test_data = _file.readlines()
    
test_data = np.array([line.strip().split() for line in test_data])
test_data[:10]

array([['podemos', 'pyodemos'],
       ['esse', 'esje'],
       ['já', 'jrá'],
       ['nosso', 'nossov'],
       ['são', 'sãêo'],
       ['dos', 'dosa'],
       ['muito', 'muifo'],
       ['imagem', 'iômagem'],
       ['sua', 'ósua'],
       ['também', 'tambéùm']], dtype='<U13')

In [12]:
X_test = test_data[:, 1]
Y_test = test_data[:, 0]

In [13]:
print(score(corrector_v1, X_test, Y_test))
print(score(corrector_v2, X_test, Y_test))

76.34%
54.84%


In [14]:
unknown_words = [true_word not in knowledge for true_word in Y_test]
print('%.2f%%' % (sum(unknown_words) * 100 / len(Y_test)))

6.99%
